In [1]:
import pandas as pd
import numpy as np


df_fit_all = pd.read_csv('3/fit_proba_modified.csv')
stationids = sorted(df_fit_all['STCD'].unique())

def compute_area_wse_curves(df_fit_all, stationids, n_points=100):
    """
    为每个测站构建 width - wse - area 曲线表。

    参数：
    - df_fit_all: DataFrame，包含每个 STCD 的拟合参数（wse0, a, b, w50, a50, w_low, w_high）
    - stationids: 测站列表
    - n_points: 拟合点数量（默认100）

    返回：
    - df_res: 拼接后的结果表，包含 STCD、width、wse、area 等列
    """
    df_res = []

    for s in stationids:
        df_fit = df_fit_all[df_fit_all['STCD'] == s]
        w_low, w_high, w50, a50 = df_fit.loc[df_fit.index[0], ['w_low', 'w_high', 'w50', 'a50']]
        wse0 = df_fit['wse0'].values
        a = df_fit['a'].values
        b = df_fit['b'].values

        w_list = np.linspace(w_low, w_high, n_points)
        h_list, hmax, hmin = [], [], []

        for x in w_list:
            heights = wse0 + a * x**b  # 所有参数组合在该宽度 x 下的预测水位
            h_list.append(np.median(heights))
            hmax.append(np.max(heights))
            hmin.append(np.min(heights))

        df_med = pd.DataFrame({
            'width': w_list,
            'wse': h_list,
            'wse_max': hmax,
            'wse_min': hmin
        })
        df_med.insert(0, 'STCD', s)

        # ------------------------
        # 面积曲线（area）计算逻辑：
        # 从 w50 处开始，向两侧进行梯形积分（面积 = ∫ h(w) dw）
        # ------------------------
        idx50 = np.searchsorted(w_list, w50)
        #w_list[idx50]实际比w50偏大，因为是右插

        # 对 w50 使用线性插值，估计 h50
        h50 = (h_list[idx50 - 1] * (w_list[idx50] - w50) +
               h_list[idx50] * (w50 - w_list[idx50 - 1])) / (w_list[idx50] - w_list[idx50 - 1])

        # 中点面积设定为 a50 + 梯形面积增量
        df_med.loc[idx50, 'area'] = a50 + 0.5 * (w50 + w_list[idx50]) * (h_list[idx50] - h50)

        # 向右（上游）积分面积（递增）
        for i in np.arange(idx50 + 1, len(w_list)):
            df_med.loc[i, 'area'] = df_med.loc[i - 1, 'area'] + \
                0.5 * (w_list[i - 1] + w_list[i]) * (h_list[i] - h_list[i - 1])

        # 向左（下游）积分面积（递减）
        for i in np.arange(idx50 - 1, -1, -1):
            df_med.loc[i, 'area'] = df_med.loc[i + 1, 'area'] - \
                0.5 * (w_list[i + 1] + w_list[i]) * (h_list[i + 1] - h_list[i])

        df_res.append(df_med)

    return pd.concat(df_res, ignore_index=True)


# ✅ 为何 area 从 idx50 分开算？
# 因为中值宽度 w50 和对应面积 a50 是已知参考点，
# 以它为基准向左右进行 "梯形积分"（近似 ∫h(w) dw）构建面积曲线，
# 可确保面积曲线在 w50 处连续、真实地反映河道断面。
# 
# 向右：逐步加面积（上游增宽）
# 向左：逐步减面积（下游变窄）
# 
# 这样 area(w) 曲线就具备了物理一致性和数值稳定性。


df_area_curve = compute_area_wse_curves(df_fit_all, stationids)

df_area_curve.to_csv('3/hypso_med_modified.csv', index=False)
#breakpoint()

In [2]:
print(df_area_curve['STCD'].nunique())

17
